In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
ratings = pd.read_csv('data/ratings.dat', sep='::',
                      names=['user_id', 'movie_id', 'rating', 'rating_timestamp'],engine="python"
                      ).sort_values("rating_timestamp") # sort by datetime
ratings.describe()

,user_id,movie_id,rating,rating_timestamp
count,888452.000000,8.884520e+05,888452.000000,8.884520e+05
mean,34879.738435,2.187479e+06,7.316825,1.459300e+09
std,20100.224164,2.025072e+06,1.853552,6.902560e+07
min,1.000000,8.000000e+00,0.000000,1.362062e+09
25%,17775.000000,7.654430e+05,6.000000,1.396187e+09
50%,34764.500000,1.714206e+06,8.000000,1.450370e+09
75%,51869.000000,2.883512e+06,9.000000,1.513955e+09
max,69324.000000,1.292071e+07,10.000000,1.600911e+09


In [3]:
movies = pd.read_csv('data/movies.dat', sep='::',
                      names=['movie_id','movie_title_year','genres'],engine="python"
                      ).sort_values("movie_title_year") # sort by movie title
movies.describe()
movies.head(10)

,movie_id,movie_title_year,genres
3975,68152,$ (1971),Comedy|Crime|Drama
11124,212555,$30 (1999),Comedy|Short
16621,1024733,$5 a Day (2008),Comedy|Drama
22588,2106284,$50K and a Call Girl: A Love Story (2014),Action|Adventure|Comedy|Drama|Romance
23431,2258233,$ellebrity (2012),Documentary
23837,2332503,&amp;Me (2013),Romance
25147,2614684,'71 (2014),Action|Drama|Thriller|War
5807,85127,'A' gai wak (1983),Action|Comedy
6642,92501,'A' gai wak 2 (1987),Action|Comedy|Crime
5259,80310,'Breaker' Morant (1980),Drama|History|War


In [ ]:
print('''Join the ratings with movies''')
movie_ratings = (ratings
                  .set_index("movie_id")
                  .join(movies.set_index("movie_id"),
                        how="left")
                 )

movie_ratings['rating_year'] = movie_ratings['rating_timestamp'].apply(lambda ts: pd.to_datetime(ts, unit='s').year)
movie_ratings.head(5)

Join the ratings with movies


In [ ]:
movie_ratings['rating_year'].describe()

In [ ]:
print('''Separate the genres string into individual category indicators through get_dummies()''')

categories = movie_ratings['genres'].str.get_dummies()
categories.head(5)

tidy_movie_ratings = (pd.concat([movie_ratings, categories], axis=1)
                       .drop(["rating_timestamp"], axis=1)
                )

tidy_movie_ratings["production_year"] = tidy_movie_ratings["movie_title_year"].str[-5:-1]
tidy_movie_ratings["movie_title"] = tidy_movie_ratings["movie_title_year"].str[:-7]

tidy_movie_ratings=tidy_movie_ratings.drop(["movie_title_year"], axis=1)

tidy_movie_ratings.reset_index(inplace=True)

tidy_movie_ratings.head(2)


In [ ]:
print('''Add a weighted popularity column for movie, which uses the following factors:
1. # of ratings 6 or higher; 2. Year of production: higher (weak) weight to recent year if available; 
3. Year of rating; recent year get higher rating  
Formula: popularity = (rating-5)*10 + (130-year_production)/5 + (mean(year_rating-2013))*9''')

#tidy_movie_ratings["popularity"]
tidy_movie_ratings.groupby("movie_id").sum().head(5)
tidy_movie_ratings['movie_id'][tidy_movie_ratings['rating']>5].value_counts()

In [ ]:
# Calculate aggregtes so we can calculate combined rating as described above
grouped_ratings=tidy_movie_ratings.groupby(
   ['movie_id']).agg(
    {
        'rating': lambda x: (x>5).sum()/10,    # conditional count
         'rating_year': lambda x: ((x-2013).mean()*8),  # recent year gets bigger value
         'production_year': lambda x: (x.astype(int)-1800).mean()/30  # really small weight
    })

grouped_ratings['combinedRating'] = grouped_ratings['rating']+grouped_ratings['rating_year']+grouped_ratings['production_year']
grouped_ratings.sort_values('combinedRating',ascending=False).head(5)


In [ ]:
grouped_ratings.describe(include='all')

In [ ]:
cols_to_add = tidy_movie_ratings.columns.difference(grouped_ratings.columns)
grouped_ratings_merged =  pd.merge(grouped_ratings, tidy_movie_ratings, 
                     left_index = True, 
                     right_on = 'movie_id', 
                     how='left',
                     suffixes=(None,"_y")).sort_values('combinedRating',ascending=False).drop_duplicates(subset=['movie_id'])

genre_column_names=movie_ratings['genres'].str.get_dummies().columns

grouped_ratings_merged.head(5)

In [ ]:
# Generate global recommendation chart:
# movie title, year of release, Genres,  Popularity Score (proprietary score)
global_chart=grouped_ratings_merged[["movie_title","production_year","genres", "combinedRating"]].head(10)
print(global_chart)
global_chart.to_csv('charts/global_ratings.csv',index=False)
#grouped_ratings_merged.columns

In [ ]:
# Generate global chart for each genre
for genre in genre_column_names:
    genre_chart=grouped_ratings_merged[grouped_ratings_merged[genre]>0][["movie_title","production_year","genres", "combinedRating"]].head(10)
    print(genre_chart)
    genre_chart.to_csv('charts/'+genre+'.csv',index=False)

In [ ]:
print('''The above concludes the generation of chart as per the project ask. Regarding If I had more time, what would I add ? 
1. A better rating formula which will would be validated through users feedback and refined
2. Join with other datasets that can help clarify rating
3. Add images for each selected movie through use of IMDB or another site data for a richer experience''')